In [202]:
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

Executing: /tmp/apt-key-gpghome.Ns3lWCtjpx/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
Executing: /tmp/apt-key-gpghome.H7w1bllBZA/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
Executing: /tmp/apt-key-gpghome.oeJYEpIEcw/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
gpg: cannot open '/dev/tty': No such device or address
gpg: [stdout]: write error: Broken pipe
gpg: filter_flush failed on c

In [203]:
!apt-get update
!apt-get install chromium chromium-driver
!pip3 install selenium
!pip install user-agent
!pip install webdriver_manager

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium import webdriver

from bs4 import BeautifulSoup
import requests

from user_agent import generate_user_agent
import random
from random import choice
from time import sleep
import gzip
import io
from webdriver_manager.chrome import ChromeDriverManager

Hit:1 http://deb.debian.org/debian buster InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 http://deb.debian.org/debian buster-updates InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:5 http://deb.debian.org/debian-security buster/updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:9 http://security.ubuntu.com/ubuntu jammy-security InRelease
Ign:10 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:11 https://r2u.stat.illinois.edu/ubuntu jammy Release
Hit:12 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:13 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:14 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acqui

In [204]:
# Initialize the Chrome driver
service = Service(executable_path=r'/usr/bin/chromedriver')

options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument('--disable-blink-features=AutomationControlled')
options.add_argument("window-size=1280,800")
options.add_argument(f"user-agent={generate_user_agent()}")

# Additional recommended options
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-infobars")
options.add_argument("--disable-extensions")
options.add_argument("--disable-gpu")
options.add_argument("--disable-popup-blocking")
options.add_argument("--incognito")
options.add_argument("--lang=en-US")  # Set language to English
options.add_argument('--disable-software-rasterizer')
options.add_argument('--log-level=3')
options.add_argument('--ignore-certificate-errors')
options.add_argument('--disable-logging')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--disable-browser-side-navigation')

# Initialize the driver
driver = webdriver.Chrome(service=service, options=options)

# Remove navigator.webdriver Flag using JavaScript
driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")

In [205]:
driver.get("https://www.goodreads.com/search?page=1&q=data+science&qid=8AxqeetoIL&search%5Bsource%5D=goodreads&search_type=books&tab=books")
soup = BeautifulSoup(driver.page_source, "html.parser")

In [206]:
first_link = 'https://www.goodreads.com' + soup.find('table',{'class':"tableList"}).find('a',{'class':'bookTitle'})['href']

In [207]:
first_page_links = soup.find('table',{'class':"tableList"}).find_all('a',{'class':'bookTitle'})
first_page_links

[<a class="bookTitle" href="/book/show/17912916-data-science-for-business?from_search=true&amp;from_srp=true&amp;qid=8AxqeetoIL&amp;rank=1" itemprop="url">
 <span aria-level="4" itemprop="name" role="heading">Data Science for Business: What You Need to Know about Data Mining and Data-Analytic Thinking</span>
 </a>,
 <a class="bookTitle" href="/book/show/17682206-data-smart?from_search=true&amp;from_srp=true&amp;qid=8AxqeetoIL&amp;rank=2" itemprop="url">
 <span aria-level="4" itemprop="name" role="heading">Data Smart: Using Data Science to Transform Information into Insight</span>
 </a>,
 <a class="bookTitle" href="/book/show/25407018-data-science-from-scratch?from_search=true&amp;from_srp=true&amp;qid=8AxqeetoIL&amp;rank=3" itemprop="url">
 <span aria-level="4" itemprop="name" role="heading">Data Science from Scratch: First Principles with Python</span>
 </a>,
 <a class="bookTitle" href="/book/show/33399049-r-for-data-science?from_search=true&amp;from_srp=true&amp;qid=8AxqeetoIL&amp;ra

In [208]:
first_page_links = soup.find_all('a',{'class':'bookTitle'})

In [209]:
first_page_links = ['https://www.goodreads.com' + link['href'] for link in first_page_links]
first_page_links

['https://www.goodreads.com/book/show/17912916-data-science-for-business?from_search=true&from_srp=true&qid=8AxqeetoIL&rank=1',
 'https://www.goodreads.com/book/show/17682206-data-smart?from_search=true&from_srp=true&qid=8AxqeetoIL&rank=2',
 'https://www.goodreads.com/book/show/25407018-data-science-from-scratch?from_search=true&from_srp=true&qid=8AxqeetoIL&rank=3',
 'https://www.goodreads.com/book/show/33399049-r-for-data-science?from_search=true&from_srp=true&qid=8AxqeetoIL&rank=4',
 'https://www.goodreads.com/book/show/17346997-doing-data-science?from_search=true&from_srp=true&qid=8AxqeetoIL&rank=5',
 'https://www.goodreads.com/book/show/35518108-machine-learning-for-absolute-beginners?from_search=true&from_srp=true&qid=8AxqeetoIL&rank=6',
 'https://www.goodreads.com/book/show/34213247-numsense-data-science-for-the-layman?from_search=true&from_srp=true&qid=8AxqeetoIL&rank=7',
 'https://www.goodreads.com/book/show/26457146-python-data-science-handbook?from_search=true&from_srp=true&q

# Prototype: Get Book Info Within The First Link

In [210]:
import time
driver.get('https://www.goodreads.com/book/show/17912916-data-science-for-business?from_search=true&from_srp=true&qid=8AxqeetoIL&rank=1')
time.sleep(3)
soup = BeautifulSoup(driver.page_source, "html.parser")

book_title = soup.find('h1',{'class':'Text Text__title1'}).text
print(book_title)
authors = soup.find('div',{'class':'ContributorLinksList'}).text #get all author names
print(authors)
stars = soup.find('div',{'class':'RatingStatistics__rating'}).text
print(stars)
n_ratings = soup.find('div',{'class':'RatingStatistics__meta'}).span.text.replace('ratings', ' ').replace('rating', ' ').strip()
print(n_ratings)
n_reviews = soup.find('div',{'class':'RatingStatistics__meta'}).find('span',{'data-testid':'reviewsCount'}).text.replace('reviews', ' ').replace('review', ' ').strip()
print(n_reviews)
overview_desc = soup.find('div', {'class':'DetailsLayoutRightParagraph__widthConstrained'}).text
print(overview_desc)
first_published = soup.find('div',{'class':'FeaturedDetails'}).findChildren()[1].text
print(first_published)

Data Science for Business: What You Need to Know about Data Mining and Data-Analytic Thinking
Foster Provost, Tom Fawcett
4.13
2,450
167
Written by renowned data science experts Foster Provost and Tom Fawcett, Data Science for Business introduces the fundamental principles of data science, and walks you through the "data-analytic thinking" necessary for extracting useful knowledge and business value from the data you collect. This guide also helps you understand the many data-mining techniques in use today. Based on an MBA course Provost has taught at New York University over the past ten years, Data Science for Business provides examples of real-world business problems to illustrate these principles. You’ll not only learn how to improve communication between business stakeholders and data scientists, but also how participate intelligently in your company’s data science projects. You’ll also discover how to think data-analytically, and fully appreciate how data science methods can supp

# Use Selenium to click on 'More Details' button, to get more details about the book

In [213]:
from selenium.common.exceptions import ElementClickInterceptedException, NoSuchElementException

# Open the webpage
driver.get(first_page_links[0])  # Replace 'YOUR_URL_HERE' with the actual URL

# Find the button using its aria-label
button = driver.find_element(By.CSS_SELECTOR, 'button[aria-label="Book details and editions"]')

# Scroll to the button
driver.execute_script("arguments[0].scrollIntoView();", button)
time.sleep(1)  # Wait for scrolling to complete

# Attempt to click the button
button_clicked = False
for _ in range(5):  # Try clicking the button up to 5 times
    try:
        button.click()
        button_clicked = True
        break  # Exit the loop if the button is clicked successfully
    except ElementClickInterceptedException:
        # Use JavaScript to click the button if intercepted
        driver.execute_script("arguments[0].click();", button)
        button_clicked = True
        break  # Exit the loop if the button is clicked successfully
    except NoSuchElementException:
        # If the button is not found, wait and retry
        time.sleep(1)

if not button_clicked:
    print("Failed to click the button after multiple attempts")
else:
    # Optionally, wait for the next elements to load or perform additional actions
    time.sleep(2)  # Increase the wait time to ensure the content loads

    # Re-parse the page to get updated HTML content
    soup = BeautifulSoup(driver.page_source, 'html.parser')

In [215]:
n_pages = soup.find('div',{'class':'EditionDetails'}).find_all('div', {'class':'TruncatedContent__text TruncatedContent__text--small'})[0].text.split(',')[0].strip()
print(n_pages)

413 pages


In [216]:
book_format = soup.find('div',{'class':'EditionDetails'}).find_all('div', {'class':'TruncatedContent__text TruncatedContent__text--small'})[0].text.split(',')[1].strip()
print(book_format)

Paperback


In [217]:
recent_publish_date = soup.find('div',{'class':'EditionDetails'}).find_all('div', {'class':'TruncatedContent__text TruncatedContent__text--small'})[1].text.split('by')[0].strip()
print(recent_publish_date)

September 17, 2013


In [218]:
publisher = soup.find('div',{'class':'EditionDetails'}).find_all('div', {'class':'TruncatedContent__text TruncatedContent__text--small'})[1].text.split('by')[1].strip()
print(publisher)

O'Reilly Media


In [219]:
isbn = soup.find('div',{'class':'EditionDetails'}).find_all('div', {'class':'TruncatedContent__text TruncatedContent__text--small'})[2].text.split(' ')[0].strip()
print(isbn)

9781449361327


In [220]:
language = soup.find('div',{'class':'EditionDetails'}).find_all('div', {'class':'TruncatedContent__text TruncatedContent__text--small'})[3].text
print(language)

English


# Get all page links

In [221]:
def getPageLinks():
  all_links = []

  for i in range(1,101):
    url = f'https://www.goodreads.com/search?page={i}&q=data+science&qid=8AxqeetoIL&search%5Bsource%5D=goodreads&search_type=books&tab=books'
    driver.get(url)
    time.sleep(1)
    soup = BeautifulSoup(driver.page_source, "html.parser")

    all_links.append(url)


    '''all_hrefs = soup.find_all('a', {'class': 'bookTitle'})
    # Find all book links on the page and append them directly to all_links
    all_links.extend(['https://www.goodreads.com' + link['href'] for link in all_hrefs])'''

  return all_links

In [222]:
all_pages = getPageLinks()

In [223]:
len(all_pages)

100

In [224]:
'''import csv
# Save the list of links to a CSV file
with open('page_links.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Page Link"])  # Write header
    for link in all_pages:
        writer.writerow([link])  # Write each link as a new row'''

'import csv\n# Save the list of links to a CSV file\nwith open(\'page_links.csv\', mode=\'w\', newline=\'\') as file:\n    writer = csv.writer(file)\n    writer.writerow(["Page Link"])  # Write header\n    for link in all_pages:\n        writer.writerow([link])  # Write each link as a new row'

In [225]:
import pandas as pd
page_links_df = pd.read_csv(r'/content/page_links.csv')

In [226]:
page_links_df
page_links_list = page_links_df['Page Link'].tolist()

# Get book links for all page links

In [227]:
def getBookLinks():
  book_links = []

  for page in page_links_list:
    driver.get(page)
    time.sleep(1)
    soup = BeautifulSoup(driver.page_source, "html.parser")

    all_hrefs = soup.find_all('a', {'class': 'bookTitle'})

    # Find all book links on the page and append them directly to all_links
    book_links.extend(['https://www.goodreads.com' + link['href'] for link in all_hrefs if 'href' in link.attrs])

  return book_links

In [228]:
all_book_links = getBookLinks()
len(all_book_links)

962

In [229]:
'''import csv
# Save the list of links to a CSV file
with open('book_links.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["book_link"])  # Write header
    for link in all_book_links:
        writer.writerow([link])  # Write each link as a new row'''

'import csv\n# Save the list of links to a CSV file\nwith open(\'book_links.csv\', mode=\'w\', newline=\'\') as file:\n    writer = csv.writer(file)\n    writer.writerow(["book_link"])  # Write header\n    for link in all_book_links:\n        writer.writerow([link])  # Write each link as a new row'

In [230]:
books_df = pd.read_csv(r'/content/book_links.csv')

In [231]:
books_list = books_df['book_link'].tolist()

In [232]:
books_list

['https://www.goodreads.com/book/show/17912916-data-science-for-business?from_search=true&from_srp=true&qid=8AxqeetoIL&rank=1',
 'https://www.goodreads.com/book/show/17682206-data-smart?from_search=true&from_srp=true&qid=8AxqeetoIL&rank=2',
 'https://www.goodreads.com/book/show/25407018-data-science-from-scratch?from_search=true&from_srp=true&qid=8AxqeetoIL&rank=3',
 'https://www.goodreads.com/book/show/33399049-r-for-data-science?from_search=true&from_srp=true&qid=8AxqeetoIL&rank=4',
 'https://www.goodreads.com/book/show/17346997-doing-data-science?from_search=true&from_srp=true&qid=8AxqeetoIL&rank=5',
 'https://www.goodreads.com/book/show/35518108-machine-learning-for-absolute-beginners?from_search=true&from_srp=true&qid=8AxqeetoIL&rank=6',
 'https://www.goodreads.com/book/show/34213247-numsense-data-science-for-the-layman?from_search=true&from_srp=true&qid=8AxqeetoIL&rank=7',
 'https://www.goodreads.com/book/show/26457146-python-data-science-handbook?from_search=true&from_srp=true&q

In [233]:
from selenium.common.exceptions import ElementClickInterceptedException, NoSuchElementException

def getBookInfo(books_list):
    book_data = []

    for book in books_list:
        driver.get(book)
        time.sleep(1)
        soup = BeautifulSoup(driver.page_source, "html.parser")

        book_info = {}

        try:
            book_info['title'] = soup.find('h1', {'class': 'Text Text__title1'}).text.strip()
        except Exception as e:
            print(f'Error scraping title for {book} : {e}')
            book_info['title'] = None
        book_title = book_info.get('title', 'unknown title')

        try:
            book_info['authors'] = soup.find('div', {'class': 'ContributorLinksList'}).text.strip()  # Get all author names
        except Exception as e:
            print(f'Error scraping authors for {book_title}: {e}')
            book_info['authors'] = None

        try:
            book_info['stars'] = soup.find('div', {'class': 'RatingStatistics__rating'}).text.strip()
        except Exception as e:
            print(f'Error scraping stars for {book_title}: {e}')
            book_info['stars'] = None

        try:
            book_info['n_ratings'] = soup.find('div', {'class': 'RatingStatistics__meta'}).span.text.replace('ratings', ' ').replace('rating', ' ').strip()
        except Exception as e:
            print(f'Error scraping n_ratings for {book_title}: {e}')
            book_info['n_ratings'] = None

        try:
            book_info['n_reviews'] = soup.find('div', {'class': 'RatingStatistics__meta'}).find('span', {'data-testid': 'reviewsCount'}).text.replace('reviews', ' ').replace('review', ' ').strip()
        except Exception as e:
            print(f'Error scraping n_reviews for {book_title}: {e}')
            book_info['n_reviews'] = None

        try:
            book_info['overview'] = soup.find('div', {'class': 'DetailsLayoutRightParagraph__widthConstrained'}).text.strip()
        except Exception as e:
            print(f'Error scraping overview for {book_title}: {e}')
            book_info['overview'] = None

        try:
            book_info['first_published'] = soup.find('div', {'class': 'FeaturedDetails'}).findChildren()[1].text.strip()
        except Exception as e:
            print(f'Error scraping first_published for {book_title}: {e}')
            book_info['first_published'] = None

        # Find the button using its aria-label
        try:
            button = driver.find_element(By.CSS_SELECTOR, 'button[aria-label="Book details and editions"]')

            # Scroll to the button
            driver.execute_script("arguments[0].scrollIntoView();", button)
            time.sleep(1)  # Wait for scrolling to complete

            # Attempt to click the button
            button_clicked = False
            for _ in range(5):  # Try clicking the button up to 5 times
                try:
                    button.click()
                    button_clicked = True
                    break  # Exit the loop if the button is clicked successfully
                except ElementClickInterceptedException:
                    # Use JavaScript to click the button if intercepted
                    driver.execute_script("arguments[0].click();", button)
                    button_clicked = True
                    break  # Exit the loop if the button is clicked successfully
                except NoSuchElementException:
                    # If the button is not found, wait and retry
                    time.sleep(1)

            if button_clicked:
                # Optionally, wait for the next elements to load or perform additional actions
                time.sleep(2)  # Increase the wait time to ensure the content loads

                # Re-parse the page to get updated HTML content
                soup = BeautifulSoup(driver.page_source, 'html.parser')

                try:
                    book_info['pages'] = soup.find('div', {'class': 'EditionDetails'}).find_all('div', {'class': 'TruncatedContent__text TruncatedContent__text--small'})[0].text.split(',')[0].strip()
                except Exception as e:
                    print(f'Error scraping pages for {book_title}: {e}')
                    book_info['pages'] = None

                try:
                    book_info['format'] = soup.find('div', {'class': 'EditionDetails'}).find_all('div', {'class': 'TruncatedContent__text TruncatedContent__text--small'})[0].text.split(',')[1].strip()
                except Exception as e:
                    print(f'Error scraping format for {book_title}: {e}')
                    book_info['format'] = None

                try:
                    book_info['recent_publish'] = soup.find('div', {'class': 'EditionDetails'}).find_all('div', {'class': 'TruncatedContent__text TruncatedContent__text--small'})[1].text.split('by')[0].strip()
                except Exception as e:
                    print(f'Error scraping recent_publish for {book_title}: {e}')
                    book_info['recent_publish'] = None

                try:
                    book_info['publisher'] = soup.find('div', {'class': 'EditionDetails'}).find_all('div', {'class': 'TruncatedContent__text TruncatedContent__text--small'})[1].text.split('by')[1].strip()
                except Exception as e:
                    print(f'Error scraping publisher for {book_title}: {e}')
                    book_info['publisher'] = None

                try:
                    book_info['isbn'] = soup.find('div', {'class': 'EditionDetails'}).find_all('div', {'class': 'TruncatedContent__text TruncatedContent__text--small'})[2].text.split(' ')[0].strip()
                except Exception as e:
                    print(f'Error scraping isbn for {book_title}: {e}')
                    book_info['isbn'] = None

                try:
                    book_info['language'] = soup.find('div', {'class': 'EditionDetails'}).find_all('div', {'class': 'TruncatedContent__text TruncatedContent__text--small'})[3].text.strip()
                except Exception as e:
                    print(f'Trying another way to extract language {book_title}: {e}')
                    try:
                        book_info['language'] = soup.find('div', {'class': 'EditionDetails'}).find_all('div', {'class': 'TruncatedContent__text TruncatedContent__text--small'})[2].text.strip()
                    except Exception as e:
                        print(f'Trying one last time to extract language {book_title}: {e}')
                        try:
                            book_info['language'] = soup.find('div', {'class': 'EditionDetails'}).find_all('div', {'class': 'TruncatedContent__text TruncatedContent__text--small'})[1].text.strip()
                        except Exception as e:
                            print(f'Error scraping language for {book_title}: {e}')
                            book_info['language'] = None

        except NoSuchElementException as e:
            print(f'Button not found for {book_title}: {e}')

        book_data.append(book_info)

    driver.quit()
    return pd.DataFrame(book_data)

In [234]:
df = getBookInfo(books_list)
df.to_csv('goodreads_DS_books.csv', index=False)

Trying another way to extract language Data Science from Scratch: First Principles with Python: list index out of range
Error scraping isbn for R for Data Science: Import, Tidy, Transform, Visualize, and Model Data: list index out of range
Trying another way to extract language R for Data Science: Import, Tidy, Transform, Visualize, and Model Data: list index out of range
Trying one last time to extract language R for Data Science: Import, Tidy, Transform, Visualize, and Model Data: list index out of range
Trying another way to extract language Machine Learning For Absolute Beginners: A Plain English Introduction (Second Edition): list index out of range
Error scraping publisher for Numsense! Data Science for the Layman: No Math Added: list index out of range
Trying another way to extract language The Art of Data Science: A Guide for Anyone Who Works with Data: list index out of range
Trying another way to extract language What Is Data Science?: list index out of range
Trying another w